**Задание 1**


Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
r = (ratings.groupby('userId')              # группируем пользователей
     .agg([min,max,len])                    # для кождого считаем мин, макс и кол-во строк
     .loc[lambda r : r.movieId.len > 100]   # отбираем тех, у кого больше 100 оценок (строк) 
     ['timestamp'][['min','max']]           # оставляем только нужные данные
     .reset_index())
r.head()

,userId,min,max
0,4,949778714,949982274
1,8,1154389340,1154474527
2,15,997937239,1469330735
3,17,1127468587,1127476640
4,19,855190091,855195373


In [7]:
avg_lt = r['max'].mean() - r['min'].mean() # т.к. среднее разностей == разности средних

In [8]:
avg_lt

40080507.44961238

In [9]:
print(f'Active user average life time is {(avg_lt / (60 * 60 * 24)):.2f} days.')

Active user average life time is 463.89 days.


In [10]:
# если нужен столбец с разностями
r['diff'] = r['max'] - r['min']
r.head()

,userId,min,max,diff
0,4,949778714,949982274,203560
1,8,1154389340,1154474527,85187
2,15,997937239,1469330735,471393496
3,17,1127468587,1127476640,8053
4,19,855190091,855195373,5282


In [11]:
r['diff'].mean() # результат тот же

40080507.4496124

**Задание 2**


Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы: таблицу с тремя типами выручки для каждого client_id без указания адреса клиента аналогичную таблицу по типам выручки с указанием адреса клиента Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [12]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [13]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [14]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [15]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [16]:
revenue = (rzd.merge(auto, on = 'client_id', how = 'outer')
              .merge(air, on = 'client_id', how = 'outer')
              .fillna(0))
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [17]:
# check
if (sum(rzd['rzd_revenue']) == sum(revenue['rzd_revenue']) and 
 sum(auto['auto_revenue']) == sum(revenue['auto_revenue']) and 
sum(air['air_revenue']) == sum(revenue['air_revenue'])):
    print("Ура! Данные в исходных и итоговой таблицах совпадают.") 
else:
    print("Упс... Данные в исходных и итоговой таблицах не совпадают.")

Ура! Данные в исходных и итоговой таблицах совпадают.


In [18]:
revenue.merge(client_base, on = 'client_id', how = 'left')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


**Задание 3**


В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

**В данном случае интересует структура таблицы с координатами. Кроме полей userid и координат, есть ли дата или время записи в таблицу данных? Если есть то берем координаты по дате визита, покупки пользователя.
Если два разных координата у одно пользователя и мы не имеем временную привязку, тогда в точности нельзя будет проставить их в таблицу по визитам, можно будет взять только самую актуальную координату (записана крайней для данного порльзователя в эту таблицу)
Из координат можно получить информацию о последовательности перемещения пользователя, как часто посещает те или иные места или как долго в них находится.**

**Решение**

1. Если координаты собирались приложением, скорее всего вместе с ними в логе есть время, когда эти координаты были записаны. Можно сравнить время, когда были записаны координаты, и время покупки/визита и определить наиболее близкие по времени. Если две координаты удалены от времени покупки на одинаковый интервал (в каких-то временных рамках, скажем, не больше 30 минут), можно взять среднее на линии между этими точками, или более позднюю координату.
Также нужно задать ограничения - когда координата не может быть привязана к покупке или визиту - какой-то разумный временной интервал. Этот интервал будет зависеть от требуемой точности - если нас интересует город - он может быть один, если улица и дом другой.


2. Если временной привязки нет и координат много, можно хранить списки координат (если нас интересует где пользователь бывает). Если координат совсем много, возможно, стоит ограничить длину этого списка - например, хранить N последних координат или хранить последнюю координату в каждом заданном временном интервале (час, день).
Также можно хранить словарь координат, где индексом может быть какая-то обобщенная метрика, связанная с координатами (город, район), а значениями либо количество координат, попадающих внутрь этого района, либо список координат внутри этого района. Это даст информацию о том где пользователь часто бывает.


3. Повторяющиеся координаты (в рамках заданной погрешности) могут быть использованы для определения мест, где пользователь проводит много времени или часто бывает (дом, работа).
Если пользователь совершает регулярные покупки/визиты из одного и того же места (в рамках погрешности), например, из дома, можно высылать ему какие-то предложения, когда он находится именно в этом месте.
Также можно сравнивать координаты разных пользователей и находить людей, которые бывают в одних и тех же местах (если есть время, то с учетом времени) - коллег, семью.
Можно сравнивать в каких местах пользователи совершают больше всего покупок/визитов - это может помочь определить либо места наиболее располагающие к покупке, либо места где есть скопления пользователей похожего профиля, максимально склонных купить то что мы предлагаем.
